In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [2]:
spark = SparkSession.\
        builder.\
        appName("book-recs").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

23/11/10 12:44:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
users_schema = StructType([
    StructField('user_id', IntegerType(), True),
    StructField('location', StringType(), True), 
    StructField('age', FloatType(), True),
    StructField('_corrupt_record', StringType(), True)
])

users_df = spark.read.csv(
    path='data/Users.csv', 
    schema=users_schema,
    mode='PERMISSIVE',
    columnNameOfCorruptRecord='_corrupt_record',
    escape='"'
).cache()

In [4]:
corrupt_user_records = users_df.filter(~users_df._corrupt_record.isNull())
corrupt_user_records.show()

+-------+----------+----+--------------------+
|user_id|  location| age|     _corrupt_record|
+-------+----------+----+--------------------+
|   null|  Location|null|User-ID,Location,Age|
| 275081|cernusco s|null|  275081,"cernusco s|
|   null|     milan|null|    , milan, italy",|
+-------+----------+----+--------------------+



In [5]:
print(f'Number of corrupt records to drop: {corrupt_user_records.count()}')
users_df = users_df.filter(users_df._corrupt_record.isNull())
users_df = users_df.drop('_corrupt_record')
users_df.unpersist();

Number of corrupt records to drop: 3


In [6]:
books_schema = StructType([
    StructField('isbn', StringType(), True),
    StructField('book_title', StringType(), True), 
    StructField('book_author', StringType(), True),
    StructField('year_of_publication', IntegerType(), True),
    StructField('publisher', StringType(), True),
    StructField('image_url_s', StringType(), True),
    StructField('image_url_m', StringType(), True),
    StructField('image_url_l', StringType(), True),
    StructField('_corrupt_record', StringType(), True)
])

books_df = spark.read.csv(
    path='data/Books.csv', 
    schema=books_schema,
    mode='PERMISSIVE',
    columnNameOfCorruptRecord='_corrupt_record',
    escape='"',
).cache()

In [7]:
corrupt_book_records = books_df.filter(~books_df._corrupt_record.isNull())
corrupt_book_records.show()

+----------+--------------------+-----------+-------------------+--------------------+--------------------+--------------------+-----------+--------------------+
|      isbn|          book_title|book_author|year_of_publication|           publisher|         image_url_s|         image_url_m|image_url_l|     _corrupt_record|
+----------+--------------------+-----------+-------------------+--------------------+--------------------+--------------------+-----------+--------------------+
|      ISBN|          Book-Title|Book-Author|               null|           Publisher|         Image-URL-S|         Image-URL-M|Image-URL-L|ISBN,Book-Title,B...|
|078946697X|DK Readers: Creat...|       2000|               null|http://images.ama...|http://images.ama...|http://images.ama...|       null|078946697X,"DK Re...|
|2070426769|Peuple du ciel, s...|       2003|               null|http://images.ama...|http://images.ama...|http://images.ama...|       null|2070426769,"Peupl...|
|0789466953|DK Readers: Crea

In [8]:
print(f'Number of corrupt records to drop: {corrupt_book_records.count()}')
books_df = books_df.filter(books_df._corrupt_record.isNull())
books_df = books_df.drop('_corrupt_record')
books_df.unpersist();

Number of corrupt records to drop: 4


In [9]:
ratings_schema = StructType([
    StructField('user_id', IntegerType(), True),
    StructField('isbn', StringType(), True), 
    StructField('book_rating', IntegerType(), True),
    StructField('_corrupt_record', StringType(), True)
])

ratings_df = spark.read.csv(
    path='data/Ratings.csv', 
    schema=ratings_schema,
    mode='PERMISSIVE',
    columnNameOfCorruptRecord='_corrupt_record',
).cache()

In [10]:
corrupt_rating_records = ratings_df.filter(~ratings_df._corrupt_record.isNull())
corrupt_rating_records.show()

+-------+----+-----------+--------------------+
|user_id|isbn|book_rating|     _corrupt_record|
+-------+----+-----------+--------------------+
|   null|ISBN|       null|User-ID,ISBN,Book...|
+-------+----+-----------+--------------------+



In [11]:
print(f'Number of corrupt records to drop: {corrupt_rating_records.count()}')
ratings_df = ratings_df.filter(ratings_df._corrupt_record.isNull())
ratings_df = ratings_df.drop('_corrupt_record')
ratings_df.unpersist();

Number of corrupt records to drop: 1


First let's check if there are duplicate values in the dataframes.

In [12]:
print(f'Dupliactes in users_df: {users_df.distinct().count() != users_df.count()}')
print(f'Dupliactes in books_df: {books_df.distinct().count() != books_df.count()}')
print(f'Dupliactes in ratings_df: {ratings_df.distinct().count() != ratings_df.count()}')

Dupliactes in users_df: False


Dupliactes in books_df: False


Dupliactes in ratings_df: False


Now let's count missing values.

In [13]:
from pyspark.sql.functions import when, count, col

for df in users_df, books_df, ratings_df:
    df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+--------+------+
|user_id|location|   age|
+-------+--------+------+
|      0|       0|110761|
+-------+--------+------+

+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+
|isbn|book_title|book_author|year_of_publication|publisher|image_url_s|image_url_m|image_url_l|
+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+
|   0|         0|          1|                  0|        2|          0|          0|          0|
+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+

+-------+----+-----------+
|user_id|isbn|book_rating|
+-------+----+-----------+
|      0|   0|          0|
+-------+----+-----------+



In [14]:
books_df.filter(books_df['book_author'].isNull()).show()

+----------+--------------------+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      isbn|          book_title|book_author|year_of_publication|           publisher|         image_url_s|         image_url_m|         image_url_l|
+----------+--------------------+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+
|9627982032|The Credit Suisse...|       null|               1995|Edinburgh Financi...|http://images.ama...|http://images.ama...|http://images.ama...|
+----------+--------------------+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+



In [15]:
no_book_author_isbns = books_df.filter(books_df['book_author'].isNull()).select('isbn').collect()
ratings_df.filter(ratings_df.isbn.isin([row[0] for row in no_book_author_isbns])).count()

1

Book with book_author missing was rated once. 

In [16]:
books_df.filter(books_df['publisher'].isNull()).show()

+----------+---------------+---------------+-------------------+---------+--------------------+--------------------+--------------------+
|      isbn|     book_title|    book_author|year_of_publication|publisher|         image_url_s|         image_url_m|         image_url_l|
+----------+---------------+---------------+-------------------+---------+--------------------+--------------------+--------------------+
|193169656X|    Tyrant Moon|Elaine Corvidae|               2002|     null|http://images.ama...|http://images.ama...|http://images.ama...|
|1931696993|Finders Keepers|Linnea Sinclair|               2001|     null|http://images.ama...|http://images.ama...|http://images.ama...|
+----------+---------------+---------------+-------------------+---------+--------------------+--------------------+--------------------+



In [17]:
no_publisher_isbns = books_df.filter(books_df['publisher'].isNull()).select('isbn').collect()
ratings_df.filter(ratings_df.isbn.isin([row[0] for row in no_publisher_isbns])).count()

2

Books with publisher missing were rated a total of two times. 

In [18]:
books_df = books_df.na.fill('Unknown')

In [19]:
books_df = books_df.drop('image_url_s', 'image_url_m', 'image_url_l')

In [ ]:
from pyspark.sql.functions import countDistinct

print('Distinct isbn values in books_df:')
books_df.agg(countDistinct(col("isbn"))).show()

print('Distinct isbn values in ratings_df:')
ratings_df.agg(countDistinct(col("isbn"))).show()

Distinct isbn values in books_df:


# Popularity-based recommender system

In [ ]:
df = ratings_df.join(users_df, on='user_id', how='left')
df = df.join(books_df, on='isbn', how='left')

In [ ]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
class PopularityBasedRecSys:
    
    def __init__(self, n_recs=5):
        self.n_recs = n_recs
        self.recs = None
        
    def fit(self, df):
#         self.recs = df.\
#                     groupBy('isbn').agg(count('isbn').alias('popularity')).\
#                     orderBy('popularity', ascending=False)
        df.createOrReplaceTempView('data')
        self.recs = spark.sql('''SELECT COUNT(isbn) AS popularity, isbn, book_title, book_author
                                 FROM data
                                 GROUP BY isbn, book_title, book_author
                                 ORDER BY COUNT(isbn) DESC''')
       
    def predict(self):
        return self.recs.limit(self.n_recs)


pop_recsys = PopularityBasedRecSys(n_recs=10)
pop_recsys.fit(df)
book_recs = pop_recsys.predict()
book_recs.show()

In [ ]:
books_df.filter(books_df.isbn == '0679781587').show()

Since 0679781587 isbn is not present in books_df, book_title and book_author will not be present in the resulting data frame of recommendations.

In [ ]:
class HighestRatedPopularityBasedRecSys(PopularityBasedRecSys):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def fit(self, df):
        df.createOrReplaceTempView('data')
        self.recs = spark.sql('''SELECT isbn, AVG(book_rating) AS popularity, book_title, book_author
                                 FROM data
                                 GROUP BY isbn, book_title, book_author
                                 ORDER BY popularity DESC''')
        
        
highest_rated_pop_recsys = HighestRatedPopularityBasedRecSys()
highest_rated_pop_recsys.fit(df)
highest_rated_book_recs = highest_rated_pop_recsys.predict()
highest_rated_book_recs.show()

In [ ]:
# class RatingWeightedPopularityBasedRecSys(PopularityBasedRecSys):
    
#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)
        
#     def fit(self, df):
#         df.createOrReplaceTempView('data')
#         self.recs = spark.sql('''SELECT isbn, COUNT(isbn) AS num_reads, ROUND(AVG(book_rating), 2) AS book_rating,\
#                                         COUNT(isbn) * AVG(book_rating) AS popularity, book_title, book_author
#                                  FROM data
#                                  GROUP BY isbn, book_title, book_author
#                                  ORDER BY popularity DESC''')

# rating_weighted_pop_recsys = RatingWeightedPopularityBasedRecSys()
# rating_weighted_pop_recsys.fit(df)
# rating_weighted_book_recs = rating_weighted_pop_recsys.predict()
# rating_weighted_book_recs.show()